### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [2]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function Y(x)
    a = 0.1836121365412579
    b = 0.2749462902187088
    c = 0.25
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [3]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [4]:
startI = -1.8
endI = -1.24798
lengthI = 100

rangeI = range( startI, endI, length = lengthI )
rangeU0 = Y( rangeI );

In [5]:
file = "output_from_function.txt"

"output_from_function.txt"

In [7]:
function firstdot( I0, U0, initial_condition_function )
    
        println("  ");flush(stdout)
        println(" >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ");flush(stdout)
       
        println("MOVE TO RIGHT")
    
        step_to_right = rangeI.step.hi
        range_to_right_border = range( I0 + step_to_right, endI, step = step_to_right )
    
        for (index_right, I0_right) in enumerate(range_to_right_border)
        
            if index_right == 1
                global initial_condition_right = initial_condition_function
            end
            println("->")
            print_output(index_right, I0_right, U0, initial_condition_right)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_right  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_right, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_right = tr[end, :]
        
            println("last point: $initial_condition_right");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_right $U0 $(λλ[1])")
            end
            separate()
        end
        println(" >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ");flush(stdout)
end

firstdot (generic function with 1 method)

In [6]:
function lastdot(I0, U0, initial_condition_function)
    
    println("  ");flush(stdout)
    println(" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ");flush(stdout)
    
    println("MOVE TO LEFT")
        
    step_to_left = rangeI.step.hi
    range_to_left_border = range( I0 - step_to_left, startI, step = -step_to_left )
        
    for (index_left, I0_left) in enumerate( range_to_left_border )
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
        end
            
        println(" <- ")
        print_output(index_left, I0_left, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0_left $U0 $(λλ[1])")
        end
        separate()
    end
    println(" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ");flush(stdout)
end

lastdot (generic function with 1 method)

In [ ]:
for (index, I0) in enumerate(rangeI)
    """
        Итерируемся по точкам c функции
    """
    if index == 1
        global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
    end
    
    U0 = Y(I0)
    
    print_output(index, I0, U0, initial_condition_function)
    
    parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
    tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
    
    initial_condition_function = tr[end, :]
    println("last point: $initial_condition_function");flush(stdout)
    λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
    println("LSE: $λλ");flush(stdout)

    open(file, "a") do io
        println(io, " $I0 $U0 $(λλ[1])")
    end
    
    println("----------------------")
    if I0 == startI
        println("  ")
        println("FIRST ITERATION");flush(stdout)
        firstdot( I0, U0, initial_condition_function )
    end
    
    if I0 == endI
        println("  ")
        println("LAST ITERATION");flush(stdout)
        lastdot(I0, U0, initial_condition_function)
    end
    
    if startI < I0 < endI
        println("  ")
        println("$index ITERATION");flush(stdout)
        lastdot(I0, U0, initial_condition_function)
        firstdot( I0, U0, initial_condition_function )
    end
    
    separate()
end

index: 1
I0: -1.8
U0: 0.34999999999999987
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
LSE: [-0.00022855632034932043, -0.555555539508975, -2.2788049535087116, -8.775049309549539]
----------------------
  
FIRST ITERATION
  
 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
MOVE TO RIGHT
->
index: 1
I0: -1.7944240404040406
U0: 0.34999999999999987
initial condition: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
last point: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.03512535069280929]
LSE: [0.001484303391490699, -0.5555555556801215, -2.321858223557142, -8.652236194996519]
_________________

->
index: 2
I0: -1.788848080808081
U0: 0.34999999999999987
initial condition: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.03512535069280929]
last point: [0.9759435544199823, 0.7366430687949771, 0.7856605876013553, 0.032568412554895

last point: [28.29861655726123, 0.4821133760128753, 0.8078020781678754, 0.013389391134611182]
LSE: [0.007573097907152367, -0.5555555553781446, -2.926397318991242, -4.9787131150527975]
_________________

->
index: 23
I0: -1.6717529292929323
U0: 0.34999999999999987
initial condition: [28.29861655726123, 0.4821133760128753, 0.8078020781678754, 0.013389391134611182]
last point: [1.3552962832380655, 0.6073759063763854, 0.8219657263345472, 0.01129353314478279]
LSE: [0.0015223754392208171, -0.5555555555318349, -2.936281446343417, -4.743891778335746]
_________________

->
index: 24
I0: -1.6661769696969728
U0: 0.34999999999999987
initial condition: [1.3552962832380655, 0.6073759063763854, 0.8219657263345472, 0.01129353314478279]
last point: [1.897485211487522, 0.7519022330348588, 0.7818140923494131, 0.010935935202837456]
LSE: [-0.0038835620880160056, -0.5555555561464718, -2.939121246598831, -4.503414429785073]
_________________

->
index: 25
I0: -1.6606010101010134
U0: 0.34999999999999987
initi

_________________

->
index: 45
I0: -1.549081818181824
U0: 0.34999999999999987
initial condition: [9.883964084344889, 0.44121052317352283, 0.8542400699058622, 8.150999111040266e-5]
last point: [9.897122860432376, 0.44083914588207007, 0.8543904527130821, 8.09068767186309e-5]
LSE: [-0.5555555555687742, -1.7207340708425378, -1.7200416882131946, -4.243148403648089]
_________________

->
index: 46
I0: -1.5435058585858645
U0: 0.34999999999999987
initial condition: [9.897122860432376, 0.44083914588207007, 0.8543904527130821, 8.09068767186309e-5]
last point: [9.910253175081097, 0.4404691688045531, 0.8545402012731571, 8.031047325368497e-5]
LSE: [-0.555555555568774, -1.7491079047738503, -1.7612579301695785, -4.248792762849031]
_________________

->
index: 47
I0: -1.537929898989905
U0: 0.34999999999999987
initial condition: [9.910253175081097, 0.4404691688045531, 0.8545402012731571, 8.031047325368497e-5]
last point: [9.923355273921622, 0.4401005798066231, 0.8546893213341025, 7.972067863834469e-5]

LSE: [-0.5555555555687541, -2.465126509077714, -2.4638378975873922, -4.363547692781525]
_________________

->
index: 68
I0: -1.4208347474747562
U0: 0.34999999999999987
initial condition: [10.17982183189432, 0.4330030149512341, 0.8575479107744479, 6.917165077693315e-5]
last point: [10.192382587369362, 0.4326610475950516, 0.857685027320737, 6.870021710460025e-5]
LSE: [-0.5555555555687541, -2.497652149032461, -2.4969357962357734, -4.368845210731313]
_________________

->
index: 69
I0: -1.4152587878787968
U0: 0.34999999999999987
initial condition: [10.192382587369362, 0.4326610475950516, 0.857685027320737, 6.870021710460025e-5]
last point: [10.204919765699357, 0.43232023926285995, 0.8578216235263332, 6.823357770280547e-5]
LSE: [-0.5555555555687532, -2.530557832005605, -2.529487521942649, -4.374128613263124]
_________________

->
index: 70
I0: -1.4096828282828373
U0: 0.34999999999999987
initial condition: [10.204919765699357, 0.43232023926285995, 0.8578216235263332, 6.823357770280547e-5]
la

last point: [10.463055810013497, 0.42541611429075976, 0.8605769400653026, 5.943394102901744e-5]
LSE: [-0.5555555555687434, -3.199565817265316, -3.1985839210848894, -4.482030132003566]
_________________

->
index: 91
I0: -1.2925876767676885
U0: 0.34999999999999987
initial condition: [10.463055810013497, 0.42541611429075976, 0.8605769400653026, 5.943394102901744e-5]
last point: [10.475116383921069, 0.4250987274032579, 0.8607030643843968, 5.9057893040741326e-5]
LSE: [-0.5555555555687433, -3.2301508018320866, -3.230063634556259, -4.487032319677627]
_________________

->
index: 92
I0: -1.287011717171729
U0: 0.34999999999999987
initial condition: [10.475116383921069, 0.4250987274032579, 0.8607030643843968, 5.9057893040741326e-5]
last point: [10.487157033449234, 0.4247823193167729, 0.8608287529252615, 5.868537291971776e-5]
LSE: [-0.5555555555687431, -3.2585218524344493, -3.2636139890399085, -4.492022922212218]
_________________

->
index: 93
I0: -1.2814357575757696
U0: 0.34999999999999987
ini

...

initial condition: [9.666980627577056, 0.4521004694748406, 0.8357643900776678, 0.00010134183860813708]
last point: [9.680934070569572, 0.45169006617689994, 0.8359437463333822, 0.00010051352834594526]
LSE: [-0.5555555555688256, -1.4660735745064954, -1.4701906114556307, -3.8445877575928855]
_________________

->
index: 34
I0: -1.5379298989899033
U0: 0.3229804517222145
initial condition: [9.680934070569572, 0.45169006617689994, 0.8359437463333822, 0.00010051352834594526]
last point: [9.694851931626568, 0.4512814141415197, 0.8361222554059626, 9.969547871721397e-5]
LSE: [-0.5555555555688247, -1.5024234121093627, -1.5061212942394915, -3.8504234752566107]
_________________

->
index: 35
I0: -1.5323539393939438
U0: 0.3229804517222145
initial condition: [9.694851931626568, 0.4512814141415197, 0.8361222554059626, 9.969547871721397e-5]
last point: [9.708734562511664, 0.45087449598655877, 0.8362999260429781, 9.888751480678051e-5]
LSE: [-0.5555555555688244, -1.5413514135432071, -1.5392507383064318,

LSE: [-0.5555555555687951, -2.239071074979962, -2.238566225800128, -3.968190278440586]
_________________

->
index: 56
I0: -1.415258787878795
U0: 0.3229804517222145
initial condition: [9.979484431340289, 0.4430757643084725, 0.8396895688953844, 8.460780259377045e-5]
last point: [9.992699485079012, 0.44270168821979283, 0.83985142490286, 8.397723671503512e-5]
LSE: [-0.5555555555687941, -2.2697072216241745, -2.2757022693958855, -3.9735891399657004]
_________________

->
index: 57
I0: -1.4096828282828355
U0: 0.3229804517222145
initial condition: [9.992699485079012, 0.44270168821979283, 0.83985142490286, 8.397723671503512e-5]
last point: [10.00588581282737, 0.44232902475598246, 0.8400126036384833, 8.335372449490182e-5]
LSE: [-0.5555555555687939, -2.3084166387797755, -2.3045780922697823, -3.978970526742065]
_________________

->
index: 58
I0: -1.404106868686876
U0: 0.3229804517222145
initial condition: [10.00588581282737, 0.44232902475598246, 0.8400126036384833, 8.335372449490182e-5]
last poi

last point: [10.276573937796343, 0.434809124250607, 0.8432510349175908, 7.171572663524878e-5]
LSE: [-0.5555555555687729, -2.995866032190753, -2.995567079534276, -4.08823916385306]
_________________

->
index: 79
I0: -1.2870117171717272
U0: 0.3229804517222145
initial condition: [10.276573937796343, 0.434809124250607, 0.8432510349175908, 7.171572663524878e-5]
last point: [10.289185483612666, 0.43446471586373503, 0.843398722279941, 7.122347864811747e-5]
LSE: [-0.5555555555687728, -3.029435239711869, -3.025795747885265, -4.093277182656235]
_________________

->
index: 80
I0: -1.2814357575757678
U0: 0.3229804517222145
initial condition: [10.289185483612666, 0.43446471586373503, 0.843398722279941, 7.122347864811747e-5]
last point: [10.301773239003232, 0.4341214765351506, 0.8435458537932791, 7.073626281592439e-5]
LSE: [-0.5555555555687726, -3.058064805286707, -3.0608053320549087, -4.098301247325348]
_________________

->
index: 81
I0: -1.2758597979798083
U0: 0.3229804517222145
initial conditi

last point: [1.9832298679694194, 0.8462667826150089, 0.6901349307245094, 0.0771896852359772]
LSE: [-3.46759116485889e-7, -0.5555555938706218, -0.9900927491713645, -8.914123623676282]
_________________

 <- 
index: 14
I0: -1.7999999999999983
U0: 0.32098194735724056
initial condition: [1.9832298679694194, 0.8462667826150089, 0.6901349307245094, 0.0771896852359772]
last point: [0.877180742289908, 0.741092265831301, 0.7514432283384187, 0.07611790320346101]
LSE: [-0.00873701272873429, -0.5555559473744612, -0.7155232424065827, -8.85245292200968]
_________________

 <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
  
 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
MOVE TO RIGHT
->
index: 1
I0: -1.7163606060606063
U0: 0.32098194735724056
initial condition: [1.0635136486557102, 0.6313379164518697, 0.7904417165149927, 0.03178339800284614]
last point: [1.0259154330437539, 0.6553604642441337, 0.785759230256048, 0.029972668217843418]
LSE: [0.001663796957765142, -0.5555555561081232, -2.1

Excessive output truncated after 524312 bytes.